# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import matplotlib.dates as dates

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')


ImportError: No module named 'mplleaflet'

In [6]:
df1 = prepare_data()
df2 = getHigsAndLowsBefore2014(df1)
df3min, df3max = getHighsAndLows2015(df1, df2)
plotData(df2, df3min, df3max)
plt.savefig('assignment2.png', format='png')



NameError: name 'pd' is not defined

In [2]:
def prepare_data():
    df1 = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
    
    # convert tenths of degrees to degrees
    df1['Temperature'] = df1['Data_Value']/10
    
    # sort by date
    df1.sort_values(by=['Date', 'Data_Value'], inplace=True)

    # Convert date column to type Date
    df1['Date'] = pd.to_datetime(df1['Date'], format='%Y-%m-%d')

    # add helper columns columns year and month-day. These are strings
    df1['Year'] = df1['Date'].dt.year
    df1['Month/Day'] = df1['Date'].dt.strftime('%m/%d') 
    
    # remove rows corresponding to 2-29 dates
    df1 = df1[df1['Month/Day'] != '02/29']
    
    # convert Month/Day from string to datetime. This will help with plotting
    # (don't worry about the 1900 year, its the relative date number we are interested in)
    df1['Month/Day'] = pd.to_datetime(df1['Month/Day'], format='%m/%d')
    
    # add a numerical representation of the date. This will help with the plotting
    #df1['numerical_date'] = df1['Month/Day'].apply(lambda x: dates.date2num(x))
    
    # get rid of Date and Data_Values rows
    #df1.drop('Data_Value', axis=1, inplace=True)
    #df1.drop('Date', axis=1, inplace=True)
    
    #print(df1)
    
    return df1

In [3]:
def getHigsAndLowsBefore2014(df):
    
    # remove the 2015 data
    df2 = df[df.Year < 2015]
    
    # get min and max based on Date, reset the index
    result = df2.groupby('Month/Day')['Temperature'].agg(['max', 'min']).reset_index()
    
    # add a numerical representation of the date. This will help with the plotting
    result['numerical_date'] = result['Month/Day'].apply(lambda x: dates.date2num(x))
    
    return result

In [4]:
def getHighsAndLows2015(df1, df2):
    
    # remove 2014 and earlier data
    df3 = df1[df1.Year == 2015]
    
    # get min and max based on Month/Dat
    df3_maxmin = df3.groupby('Month/Day')['Temperature'].agg(['max', 'min']).reset_index()
    
    # merge df2 with the maxmin of df3
    result = pd.merge(df2, df3_maxmin, how='outer', left_on='Month/Day', right_on='Month/Day')
    
    # compare 2015 (min_y, max_y) with 2005-2014 (min_x, max_x) 
    result['Min_in_2015'] = result['min_y'] < result['min_x']
    result['Max_in_2015'] = result['max_y'] > result['max_x']
    
    # create min and max df
    result_min = result[result.Min_in_2015 == True].copy() 
    result_max = result[result.Max_in_2015 == True].copy()
    result_min.rename(columns={'min_y':'min'}, inplace=True)
    result_max.rename(columns={'max_y':'max'}, inplace=True)
    
    # add a numerical representation of the date. This will help with the plotting
    result_min['numerical_date'] = result_min['Month/Day'].apply(lambda x: dates.date2num(x))
    result_max['numerical_date'] = result_max['Month/Day'].apply(lambda x: dates.date2num(x))

    return result_min, result_max

In [5]:
def plotData(df2, df3min, df3max):
    %matplotlib notebook
    
    # create new figure
    plt.figure()
    
    # plot record low and high data from 2005-2014
    plt.plot(df2['numerical_date'], df2['max'], color='r', lw=1, 
             label='record high 2004-2015', zorder=1)
    plt.plot(df2['numerical_date'], df2['min'], color='b', lw=1, 
             label='record low 2004-2014', zorder=1)
    


    # add scatterplot for 2015 data where it was a high or low
    plt.scatter(df3max['numerical_date'], df3max['max'], color='k', marker='o', s=10, 
                label='record breaking high in 2015', zorder=3 )  
    plt.scatter(df3min['numerical_date'], df3min['min'], color='darkviolet', marker='o', s=10, 
                label='record breaking low in 2015', zorder=3)
    
    # fill in the space between the two lines
    plt.gca().fill_between(df2['numerical_date'], df2['min'], df2['max'], facecolor='lightgrey',
                          zorder=2)
    
    # Modify the axes and legend
    ax = plt.gca()
    
    # Set x axis format (month), remove the last month
    ax.xaxis.set_major_formatter(dates.DateFormatter('%b'))
    ax.xaxis.set_major_locator(dates.MonthLocator())
    loc, labels = plt.xticks()
    loc = loc[:-1]
    ax.set_xticks(loc)
    
    # Set axes labels and title
    plt.xlabel('Month')
    plt.ylabel('Temperature (Celcius)')
    plt.title('Record Daily Temperatures for Ann Arbor, Michigan')
    
    # cosmetics
    
    # remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # add legend
    plt.subplots_adjust(bottom=0.15)
    plt.legend(loc='best', prop={'size': 8})
    
    return

